# randamforest_pyspark

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('randomforest_pyspark').getOrCreate()

In [17]:
from pyspark.sql.types import *                                                  

labels = StructType([                                                            

    StructField('failure', DoubleType(), False),                                 
    StructField('weight', DoubleType(), False),                                  
    StructField('vel', DoubleType(), False),                                 
    StructField('mean1k_max', DoubleType(), False),                                  
    StructField('std1k_max', DoubleType(), False),
    StructField('max1k_max', DoubleType(), False),                                 
    StructField('mean512_max', DoubleType(), False),                                  
    StructField('std512_max', DoubleType(), False),                                 
    StructField('max512_max', DoubleType(), False),                                  
    StructField('mean256_max', DoubleType(), False),
    StructField('std256_max', DoubleType(), False),                                 
    StructField('max256_max', DoubleType(), False),                                  
    StructField('mean128_max', DoubleType(), False),                                 
    StructField('std128_max', DoubleType(), False),                                  
    StructField('max128_max', DoubleType(), False),
    StructField('mean64_max', DoubleType(), False),                                 
    StructField('std64_max', DoubleType(), False),                                  
    StructField('max64_max', DoubleType(), False),                                 
    StructField('mean32_max', DoubleType(), False),                                  
    StructField('std32_max', DoubleType(), False),
    StructField('max32_max', DoubleType(), False),                                 
    StructField('mean16_max', DoubleType(), False),                                  
    StructField('std16_max', DoubleType(), False),                                 
    StructField('max16_max', DoubleType(), False)
    ])         

In [18]:
#read csv
df=spark.read.csv("feature.csv",schema=labels,header="True",sep=",")   
df.show(5)

+-------+--------+-----+----------+---------+---------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+----------+---------+---------+----------+---------+---------+----------+---------+---------+
|failure|  weight|  vel|mean1k_max|std1k_max|max1k_max|mean512_max|std512_max|max512_max|mean256_max|std256_max|max256_max|mean128_max|std128_max|max128_max|mean64_max|std64_max|max64_max|mean32_max|std32_max|max32_max|mean16_max|std16_max|max16_max|
+-------+--------+-----+----------+---------+---------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+----------+---------+---------+----------+---------+---------+----------+---------+---------+
|    0.0|40160.16|24.94|     58.91|   166.26|  2978.29|      37.73|     64.66|   1155.34|       11.4|      21.6|    452.41|       3.97|      7.36|    102.91|      5.12|     11.9|   173.47|      15.4|     54.5|   872.63|     38.03|    177.0|   3582

In [19]:
#make pipeline   
# fearure assemble
from pyspark.ml.feature import VectorAssembler
featurecreator = VectorAssembler(inputCols=['weight', 'vel', 'mean1k_max', 'std1k_max', 'max1k_max', 'mean512_max', 'std512_max', 
                                          'max512_max', 'mean256_max', 'std256_max','max256_max', 'mean128_max', 'std128_max', 'max128_max', 
                                          'mean64_max', 'std64_max','max64_max', 'mean32_max', 'std32_max', 'max32_max', 'mean16_max', 
                                          'std16_max', 'max16_max'], outputCol="features")
#df = df_assembler.transform(df)


#standardization
from pyspark.ml.feature import StandardScaler                                       
scaler = StandardScaler(inputCol='features', outputCol= 'scaledFeatures')           

#randomforest
from pyspark.ml.classification import RandomForestClassifier                        
randamf = RandomForestClassifier(featuresCol = 'scaledFeatures', labelCol = 'failure',numTrees=50)     

#pipeline featurecreator + scaler + randaomforest
from pyspark.ml.pipeline import Pipeline                                            
pipeline = Pipeline(stages=[featurecreator,scaler,randamf])     

In [20]:
#trainiing
(train, test) = df.randomSplit([0.8, 0.2])   
model = pipeline.fit(train)     

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator                     
predict = model.transform(test)                                                         
evaluator = MulticlassClassificationEvaluator(labelCol='failure', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predict)                                                  
print("Accuracy = %g" % (accuracy))  

Accuracy = 0.814815
